In [1]:
import numpy as np
import pandas as pd

from warnings import filterwarnings

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler, normalize, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold

filterwarnings('ignore')

In [2]:
CONFIG = {
    'TRAIN_PATH': '/kaggle/input/datathon-entel-2022-reto2/train.csv',
    'TEST_PATH': '/kaggle/input/datathon-entel-2022-reto2/test.csv',
    'SAMPLE_SUBMISSION': '/kaggle/input/datathon-entel-2022-reto2/test_sample.csv'
}

df_train = pd.read_csv(CONFIG['TRAIN_PATH'])
df_test = pd.read_csv(CONFIG['TEST_PATH'])
df_sub = pd.read_csv(CONFIG['SAMPLE_SUBMISSION'])

In [3]:
def featuring(df):
    
    df = pd.concat([df_train.iloc[:, :5], df], axis=1)
        
    df_z_punto_venta = df.groupby(['Z_PUNTO_VENTA'])[df.iloc[:, 5:].columns].transform('max')
    df_z_modelo = df.groupby(['Z_MODELO'])[df.iloc[:, 5:].columns].transform('max')
    df_z_gama = df.groupby(['Z_GAMA'])[df.iloc[:, 5:].columns].transform('max')
    df_z_marca = df.groupby(['Z_MARCA'])[df.iloc[:, 5:].columns].transform('max')
    df_z_departamento = df.groupby(['Z_DEPARTAMENTO'])[df.iloc[:, 5:].columns].transform('max')
    
    df_z_s_punto_venta = df.groupby(['Z_PUNTO_VENTA'])[df.iloc[:, 5:].columns].transform('sum')
    df_z_s_modelo = df.groupby(['Z_MODELO'])[df.iloc[:, 5:].columns].transform('sum')
    df_z_s_gama = df.groupby(['Z_GAMA'])[df.iloc[:, 5:].columns].transform('sum')
    df_z_s_marca = df.groupby(['Z_MARCA'])[df.iloc[:, 5:].columns].transform('sum')
    df_z_s_departamento = df.groupby(['Z_DEPARTAMENTO'])[df.iloc[:, 5:].columns].transform('sum')
    
    df_b_punto_venta = df['Z_PUNTO_VENTA'].apply(lambda x: 1 if x in 
                                                     ['da45328ba820604eb99694768f2a430cd933d161601dcb8491b4a9b555232c59',
                                                      'e1f2d2708f545ddc1d7266ba0cc5ccc88147b77fdf3450e68a974e93018ecf60'] else 0)
    df_b_departameto = df['Z_DEPARTAMENTO'].apply(lambda x: 1 if x  in 
                                                    ['d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589'] else 0)
    
    Z_MARCA = df['Z_MARCA'].replace(df['Z_MARCA'].value_counts(normalize=True).to_dict())
    Z_GAMA = df['Z_GAMA'].replace(df['Z_GAMA'].value_counts(normalize=True).to_dict())
    Z_MODELO = df['Z_MODELO'].replace(df['Z_MODELO'].value_counts(normalize=True).to_dict())
    Z_DEPARTAMENTO = df['Z_DEPARTAMENTO'].replace(df['Z_DEPARTAMENTO'].value_counts(normalize=True).to_dict())
    Z_PUNTO_VENTA = df['Z_PUNTO_VENTA'].replace(df['Z_PUNTO_VENTA'].value_counts(normalize=True).to_dict())
        
    df_z = pd.concat([df_z_punto_venta,
                      df_z_modelo, 
                      df_z_gama,
                      df_z_marca,
                      df_z_departamento,
                      
                      df_z_s_punto_venta,
                      df_z_s_modelo,
                      df_z_s_gama,
                      df_z_s_marca,
                      df_z_s_departamento,

                      df_b_punto_venta,
                      df_b_departameto,
                      
                      Z_MARCA,
                      Z_GAMA,
                      Z_MODELO,
                      Z_DEPARTAMENTO,
                      Z_PUNTO_VENTA
                     ], axis=1).T.reset_index(drop=True).T
    
    df['M_MAX_VALUE'] = df.iloc[:, 5:].max(axis=1)
    df['M_SUM_VALUE'] = df.iloc[:, 5:].sum(axis=1)
    df['M_STD_VALUE'] = df.iloc[:, 5:].std(axis=1)
    df['M_MEAN_VALUE'] = df.iloc[:, 5:].mean(axis=1)
    
    df['M_TOTAL_VALUE'] = df['M_SUM_VALUE'].apply(lambda x: 1 if x > 0 else 0)
    
    df['M_COUNT'] = df.iloc[:, 5:].stack().apply(lambda x: x if x > 0 else np.nan).unstack(level=1).count(axis=1)
    
    df[df.iloc[:, 5:15].columns] = df.iloc[:, 5:15].stack().apply(lambda x: x if x < 100 else 100).unstack(level=1)
        
    df = df.drop(df.columns[:5], axis=1)
    
    df = pd.concat([df, df_z], axis=1)
    
    return df.T.reset_index(drop=True).T

In [4]:
x_train = pd.concat([
    featuring(df_train.iloc[:, 5:15]),
    featuring(df_train.iloc[:, 15:25]),
    featuring(df_train.iloc[:, 25:35]),
    featuring(df_train.iloc[:, 35:45]),
], axis=0).reset_index(drop=True)

y_train = pd.concat([
    pd.DataFrame(df_train.iloc[:, 15:25].values),
    pd.DataFrame(df_train.iloc[:, 25:35].values),
    pd.DataFrame(df_train.iloc[:, 35:45].values),
    pd.DataFrame(df_train.iloc[:, 45:55].values)
], axis=0).reset_index(drop=True)


x_test = featuring(df_train.iloc[:, 45:55])

In [5]:
print(f'X TRAIN SHAPE: {x_train.shape}')
print(f'Y TRAIN SHAPE: {y_train.shape}')
print(f'X TEST SHAPE: {x_test.shape}')

X TRAIN SHAPE: (188692, 123)
Y TRAIN SHAPE: (188692, 10)
X TEST SHAPE: (47173, 123)


In [6]:
def data_to_models(n_t):
    
    correlated_features = set()
    correlation_matrix = x_train.iloc[:, n_t:].corr()

    for i in range(len(correlation_matrix .columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > 0.95:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
                
    x_train_model = x_train.drop(labels=correlated_features, axis=1)
    x_test_model = x_test.drop(labels=correlated_features, axis=1)

    print(f'TRAIN SHAPE: {x_train_model.shape}')
    
    x_train_model = x_train
    x_test_model = x_test
    
    sc = StandardScaler()

    x_train_model = sc.fit_transform(x_train_model)
    x_test_model = sc.transform(x_test_model)
    
    return x_train_model, x_test_model

In [7]:
n = 10

In [8]:
x_train_lstm, x_test_lstm = data_to_models(n)
x_train_mlp, x_test_mlp = data_to_models(0)

y_train = y_train.stack().apply(lambda x: x if x < 100 else 100).unstack(level=1).values

TRAIN SHAPE: (188692, 61)
TRAIN SHAPE: (188692, 61)


## MODEL

In [9]:
def LSTM(l_features, l_extras_features):
    
    features = keras.layers.Input(shape=l_features)
    tabular = keras.layers.Input(shape=l_extras_features)

    out_features = keras.layers.LSTM(250, return_sequences=True)(features)
    out_features = keras.layers.Dropout(0.2)(out_features)
    out_features = keras.layers.LSTM(150, return_sequences=True)(out_features)
    out_features = keras.layers.Dropout(0.2)(out_features)
    out_features = keras.layers.LSTM(100)(out_features)
    out_features = keras.layers.Flatten()(out_features)

    out_features = keras.layers.Dense(50, activation='linear')(out_features)
    out_features = keras.layers.Dropout(0.2)(out_features)
    out_features = keras.layers.Dense(32, activation='linear')(out_features)

    for n_hidden in [512, 256, 128, 64, 32]:
        out_tabular = keras.layers.Dense(n_hidden, activation='relu')(tabular)
        out_tabular = keras.layers.BatchNormalization()(out_tabular)
        out_tabular = keras.layers.Dropout(0.2)(out_tabular)

    out = tf.keras.layers.Multiply()([out_features, out_tabular])
    out = keras.layers.Dense(10, activation='relu')(out)

    model = keras.Model(inputs = [features, tabular], outputs = out)

    mse = tf.keras.losses.MeanSquaredError()
    rmse = tf.keras.metrics.RootMeanSquaredError()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0004), loss=mse, metrics=[rmse])

    return model


def MLP(l_extras_features):
    
    tabular = keras.layers.Input(shape=l_extras_features)

    for n_hidden in [512, 256, 128, 64, 32]:
        out_tabular = keras.layers.Dense(n_hidden, activation='relu')(tabular)
        out_tabular = keras.layers.BatchNormalization()(out_tabular)
        out_tabular = keras.layers.Dropout(0.2)(out_tabular)

    out = keras.layers.Dense(10, activation='relu')(out_tabular)

    model = keras.Model(inputs = [tabular], outputs = out)

    mse = tf.keras.losses.MeanSquaredError()
    rmse = tf.keras.metrics.RootMeanSquaredError()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0004), loss=mse, metrics=[rmse])

    return model

In [10]:
def training_LSTM():

    EPOCH = 1000
    BATCH_SIZE = 512

    models = []

    # kf = GroupKFold(n_splits=5)
    # kf = StratifiedKFold(n_splits=5)
    kf = KFold(n_splits=5, shuffle=True, random_state=2022)

    for fold, (train_idx, val_idx) in enumerate(kf.split(x_train_lstm, y_train)):

        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)

        X_train, X_valid = x_train_lstm[train_idx], x_train_lstm[val_idx]
        
        x_t_features, x_v_features = X_train[:, :n], X_valid[:, :n]
        x_t_features = np.reshape(x_t_features, (x_t_features.shape[0], 1, x_t_features.shape[1]))
        x_v_features = np.reshape(x_v_features, (x_v_features.shape[0], 1, x_v_features.shape[1]))
        
        x_t_extras, x_v_extras = X_train[:, n:], X_valid[:, n:]
        
        Y_train, Y_valid = y_train[train_idx], y_train[val_idx]

        l_fet = x_t_features.shape[-2:]
        l_ext = x_t_extras.shape[-1]

        model = LSTM(l_fet, l_ext)

        es = keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error', 
                                           min_delta=1e-05,
                                           patience=30,
                                           verbose=1, 
                                           mode='min', 
                                           restore_best_weights=True)
        plateau = keras.callbacks.ReduceLROnPlateau(monitor='val_root_mean_squared_error',
                                                    factor=0.1,
                                                    patience=10,
                                                    verbose=1,
                                                    min_lr=5e-7, 
                                                    mode='min')

        model.fit([x_t_features, x_t_extras], Y_train,
                  validation_data=([x_v_features, x_v_extras], Y_valid),
                  epochs=EPOCH,
                  batch_size=BATCH_SIZE,
                  callbacks = [es, plateau],
                  verbose=1)

        eval_model(model, [x_v_features, x_v_extras], Y_valid)

        models.append(model)

    return models

def training_MLP():

    EPOCH = 1000
    BATCH_SIZE = 512

    models = []

    # kf = GroupKFold(n_splits=5)
    # kf = StratifiedKFold(n_splits=5)
    kf = KFold(n_splits=5, shuffle=True, random_state=2022)

    for fold, (train_idx, val_idx) in enumerate(kf.split(x_train_mlp, y_train)):

        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)

        X_train, X_valid = x_train_mlp[train_idx], x_train_mlp[val_idx]        
        Y_train, Y_valid = y_train[train_idx], y_train[val_idx]

        l_ext = X_train.shape[-1]

        model = MLP(l_ext)

        es = keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error', 
                                           min_delta=1e-05,
                                           patience=30,
                                           verbose=1, 
                                           mode='min', 
                                           restore_best_weights=True)
        plateau = keras.callbacks.ReduceLROnPlateau(monitor='val_root_mean_squared_error',
                                                    factor=0.1,
                                                    patience=10,
                                                    verbose=1,
                                                    min_lr=5e-7, 
                                                    mode='min')

        model.fit([X_train], Y_train,
                  validation_data=([X_valid], Y_valid),
                  epochs=EPOCH,
                  batch_size=BATCH_SIZE,
                  callbacks = [es, plateau],
                  verbose=1)

        eval_model(model, [X_valid], Y_valid)

        models.append(model)

    return models


def eval_model(model, x_valid, y_valid):
    preds = pd.DataFrame(np.round(model.predict(x_valid)).astype('int32')).stack().reset_index(drop=True)
    y_valid = pd.DataFrame(y_valid).stack().reset_index(drop=True)
    print(f' RMSE --> {mean_squared_error(y_valid, preds, squared=False)}')

In [11]:
models_LSTM = training_LSTM()

--------------- > Fold 1 < ---------------


2022-08-06 11:23:03.243938: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-06 11:23:03.384199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-06 11:23:03.384964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-06 11:23:03.386296: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/1000


2022-08-06 11:23:12.311003: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


295/295 [==============================] - 9s 11ms/step - loss: 10.2592 - root_mean_squared_error: 3.2030 - val_loss: 7.4696 - val_root_mean_squared_error: 2.7331
Epoch 2/1000
295/295 [==============================] - 2s 8ms/step - loss: 7.9507 - root_mean_squared_error: 2.8197 - val_loss: 6.7873 - val_root_mean_squared_error: 2.6052
Epoch 3/1000
295/295 [==============================] - 2s 8ms/step - loss: 7.6503 - root_mean_squared_error: 2.7659 - val_loss: 7.2593 - val_root_mean_squared_error: 2.6943
Epoch 4/1000
295/295 [==============================] - 2s 8ms/step - loss: 7.3863 - root_mean_squared_error: 2.7178 - val_loss: 6.3545 - val_root_mean_squared_error: 2.5208
Epoch 5/1000
295/295 [==============================] - 2s 8ms/step - loss: 7.0181 - root_mean_squared_error: 2.6492 - val_loss: 6.2060 - val_root_mean_squared_error: 2.4912
Epoch 6/1000
295/295 [==============================] - 3s 11ms/step - loss: 7.0427 - root_mean_squared_error: 2.6538 - val_loss: 6.2901 - va

In [12]:
models_MLP = training_MLP()

--------------- > Fold 1 < ---------------
Epoch 1/1000
295/295 [==============================] - 2s 3ms/step - loss: 11.8263 - root_mean_squared_error: 3.4389 - val_loss: 9.2266 - val_root_mean_squared_error: 3.0375
Epoch 2/1000
295/295 [==============================] - 1s 4ms/step - loss: 8.6905 - root_mean_squared_error: 2.9480 - val_loss: 7.2845 - val_root_mean_squared_error: 2.6990
Epoch 3/1000
295/295 [==============================] - 1s 3ms/step - loss: 7.8534 - root_mean_squared_error: 2.8024 - val_loss: 6.5383 - val_root_mean_squared_error: 2.5570
Epoch 4/1000
295/295 [==============================] - 1s 3ms/step - loss: 7.5217 - root_mean_squared_error: 2.7426 - val_loss: 6.3150 - val_root_mean_squared_error: 2.5130
Epoch 5/1000
295/295 [==============================] - 1s 3ms/step - loss: 7.3622 - root_mean_squared_error: 2.7133 - val_loss: 6.1926 - val_root_mean_squared_error: 2.4885
Epoch 6/1000
295/295 [==============================] - 1s 3ms/step - loss: 7.1726 - r

In [13]:
preds_LSTM = []

x_test_features = x_test_lstm[:, :n]
x_test_features = np.reshape(x_test_features, (x_test_features.shape[0], 1, x_test_features.shape[1]))

x_test_extras = x_test_lstm[:, n:]

for model in models_LSTM:
    _pred = model.predict([x_test_features, x_test_extras])
    preds_LSTM.append(_pred)

In [14]:
preds_MLP = []

for model in models_MLP:
    _pred = model.predict([x_test_mlp])
    preds_MLP.append(_pred)

In [15]:
pred_sub_LSTM = (preds_LSTM[0] + preds_LSTM[1] + preds_LSTM[2] + preds_LSTM[3] + preds_LSTM[4]) / 5
pred_sub_MLP = (preds_MLP[0] + preds_MLP[1] + preds_MLP[2] + preds_MLP[3] + preds_MLP[4]) / 5

In [28]:
pred_sub = pred_sub_LSTM #  * 0.7 + pred_sub_MLP * 0.3

In [29]:
df_submission = pd.merge(df_train.iloc[:, :5], pd.DataFrame(pred_sub), how='inner', left_index=True, right_index=True)
df_submission = df_submission.rename(columns={
    0: 'SEMANA_51', 
    1: 'SEMANA_52',
    2: 'SEMANA_53',
    3: 'SEMANA_54',
    4: 'SEMANA_55',
    5: 'SEMANA_56',
    6: 'SEMANA_57',
    7: 'SEMANA_58',
    8: 'SEMANA_59',
    9: 'SEMANA_60'
})

df_submission['BASE_ID'] = df_submission['Z_MODELO'].astype(str) + '|' + df_submission['Z_PUNTO_VENTA'].astype(str) + '|' + df_submission['Z_GAMA'].astype(str)
df_submission = df_submission.iloc[:, 5:]
df_submission = df_submission.set_index('BASE_ID').stack().to_frame().reset_index()
df_submission['BASE_ID'] = df_submission['BASE_ID'].astype(str) + '|' + df_submission['level_1'].astype(str)
df_submission = df_submission.drop(['level_1'], axis=1)
df_submission.columns = ['ID', 'Demanda']

In [39]:
df_submission.to_csv('entel_v5.1.csv', index=False)

In [31]:
np.round(df_submission.Demanda).value_counts(normalize=True)

0.0     0.842058
1.0     0.086950
2.0     0.027507
3.0     0.013393
4.0     0.007871
          ...   
90.0    0.000002
89.0    0.000002
85.0    0.000002
62.0    0.000002
72.0    0.000002
Name: Demanda, Length: 100, dtype: float64

In [37]:
df_submission.Demanda.quantile(0.9999)

73.93909945524175

In [33]:
pd.DataFrame(y_train).stack().value_counts(normalize=True)

0     0.837697
1     0.071655
2     0.030250
3     0.015947
4     0.009822
        ...   
89    0.000007
98    0.000006
86    0.000006
93    0.000005
92    0.000005
Length: 101, dtype: float64

In [38]:
pd.DataFrame(y_train).stack().quantile(0.9999)

100.0

<a href='./entel_v5.1.csv'>download</a>